# Story 6 : What Is The State of Food Security and Nutrition in the US

The United Nations Food and Agriculture Organization publication, The State of Food Security and Nutrition in the World 2022 (https://www.fao.org/documents/card/en/c/cc0639en) might lead one to the conclusion that it's an elsewhere problem. That the people who are suffering malnutrition and starvation are "elsewhere", not in our backyard. For this assignment you will need to take a closer look here at home (the US). Are US children suffering these plights?

Notes:

You may use ANY graphical package that you find useful including PowerBI, Tableau, etc.
You will need to locate and source data that reflects food security and nutrition by state broken down by men, women, children and by age groups.
Your analysis should demonstrate correlations that exist between level of poverty and food insecurity, malnutrition and starvation.
Your data and analysis should also indicate what happens to the children as they mature into adults. Will they become fully functional citizens or will they require continued support?
Your data visualizations need to tell the story for a U.S. Senator that you were lobbying to address the issue of food insecurity in the US.

- https://www.census.gov/data/datasets/time-series/demo/cps/cps-supp_cps-repwgt/cps-food-security.html#cpssupps 
- https://www.ers.usda.gov/data-products/food-security-in-the-united-states/

In [2]:
# load packages
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec, GridSpecFromSubplotSpec
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes